# Working with Azure Machine Learning Studio engine

This notebook shows how to monitor the model deployed on Azure Machine Learning Studio using Watson OpenScale python sdk.

**Note:** This notebook works correctly with kernel **`IBM Runtime 22.1 on Python 3.9 XS`** if using IBM Watson Studio or else use standard Python 3.9 runtime.

Contents

1. [Setup](#setup)
2. [Binding machine learning engine](#binding)
3. [Subscriptions](#subsciption)
4. [Performance monitor, scoring and payload logging](#scoring)
5. [Quality monitor and feedback logging](#feedback)
6. [Fairness, Drift monitor and explainability](#monitors)

<a id="setup"></a>
# 1. Setup

### Sample model creation using [Azure Machine Learning Studio](https://studio.azureml.net)

- Copy [credit risk experiment](https://gallery.cortanaintelligence.com/Experiment/German-credit-risk-created-on-1-9-2019) from Azure ML Studio Gallery 
- Run experiment to train a model
- Create (deploy) web service (new)

**NOTE:** Classic web services are not supported.

### Installation and authentication

In [ ]:
!pip install --upgrade --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1

**Action:** Restart the kernel (Kernel->Restart)

#### ACTION: Get Watson OpenScale `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [ ]:
CLOUD_API_KEY = '***'

In [ ]:
DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

In [ ]:
SCHEMA_NAME = 'azure_dm'

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment if using IAM token
wos_client = APIClient(authenticator=authenticator)
wos_client.version

#### Download and preview training data set

In [ ]:
!rm -rf credit_risk_training.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/credit_risk_training.csv"

In [ ]:
import pandas as pd

training_data_pd = pd.read_csv(
        "credit_risk_training.csv",
        dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})

In [ ]:
training_data_pd.head()

## Save training data to IBM Cloud Object Storage

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit getting started with COS tutorial. You can find COS_API_KEY_ID and COS_RESOURCE_CRN variables in Service Credentials in menu of your COS instance. Used COS Service Credentials must be created with Role parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription. COS_ENDPOINT variable can be found in Endpoint field of the menu.

In [ ]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [ ]:
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "***" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [ ]:
BUCKET_NAME = "***" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training.csv"

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [ ]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

### DataMart setup

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

<a id="binding"></a>
# 2. Bind machine learning engines

### Bind  `Azure` machine learning studio

Provide credentials using following fields:
- `client_id`
- `client_secret`
- `subscription_id`
- `tenant`

In [ ]:
AZURE_ENGINE_CREDENTIALS = {
    "client_id": "***",
    "client_secret": "***",
    "subscription_id": "***",
    "tenant": "***"
}

In [ ]:
SERVICE_PROVIDER_NAME = "Azure Machine Learning"
SERVICE_PROVIDER_DESCRIPTION = "Added by Azure tutorial WOS notebook."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.AZURE_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=AzureCredentials(
            subscription_id= AZURE_ENGINE_CREDENTIALS['subscription_id'], 
            client_id = AZURE_ENGINE_CREDENTIALS['client_id'], 
            client_secret= AZURE_ENGINE_CREDENTIALS['client_secret'],
            tenant = AZURE_ENGINE_CREDENTIALS['tenant']
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
wos_client.service_providers.show()

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
asset_deployment_details

In [ ]:
## select your model deployment GUID and get model asset details

deployment_id='***'
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break

### sample scoring

In [ ]:
import requests
import time
import json

scoring_url = model_asset_details['entity']['scoring_endpoint']['url']

data = {
            "Inputs": {
                "input1":
                    [{'CheckingStatus': 'no_checking',
                      'LoanDuration': 19,
                      'CreditHistory': 'credits_paid_to_date',
                      'LoanPurpose': 'business',
                      'LoanAmount': 1867,
                      'ExistingSavings': '100_to_500',
                      'EmploymentDuration': '1_to_4',
                      'InstallmentPercent': 4,
                      'Sex': 'male',
                      'OthersOnLoan': 'none',
                      'CurrentResidenceDuration': 3,
                      'OwnsProperty': 'car_other',
                      'Age': 42,
                      'InstallmentPlans': 'stores',
                      'Housing': 'own',
                      'ExistingCreditsCount': 2,
                      'Job': 'unskilled',
                      'Dependents': 1,
                      'Telephone': 'none',
                      'ForeignWorker': 'yes'}]
            },
            "GlobalParameters": {
            }
        }

body = str.encode(json.dumps(data))


token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = model_asset_details['entity']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

<a id="subsciption"></a>
# 3. Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [ ]:
wos_client.subscriptions.show()

In [ ]:
feature_columns = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker']
categorical_columns = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']

In [ ]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointCredentialsAzureScoringEndpointCredentials,ScoringEndpointRequest
def get_output_schema_fields_for_azure(azure_ouput_schema, target_field, target_role = "decoded-target", field_type = str, is_nullable = True):
        fields = []
        found = False
        for field in azure_ouput_schema['fields']:
            if field['name']==target_field:
                metadata = {}
                if "metadata" in field:
                    metadata = field["metadata"]
                metadata["modeling_role"]=target_role    
                field["metadata"] = metadata
                found = True
    
            spark_field = SparkStructFieldObject(
                name=field['name'],
                type=field['type'],
                nullable=field['nullable'],
            )
            if "metadata" in field:
                spark_field.metadata = field["metadata"]
                
            fields.append(spark_field)   
        
        if found is False:
            metadata = {}
            metadata["modeling_role"] = target_role
            fields.append(SparkStructFieldObject(
                name=target_field,
                type=field_type,
                nullable=is_nullable,
                metadata = metadata
            )) 
        return fields

In [ ]:
azure_asset = Asset(
            asset_id=model_asset_details["entity"]["asset"]["asset_id"],
            name=model_asset_details["entity"]["asset"]["name"],
            url=model_asset_details["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION,
        )

In [ ]:
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']
scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url']
                                          ,request_headers = model_asset_details['entity']['scoring_endpoint']['request_headers'],
                                          credentials= ScoringEndpointCredentialsAzureScoringEndpointCredentials(model_asset_details['entity']['scoring_endpoint']['credentials']['token']) )

deployment = AssetDeploymentRequest(
    deployment_id=model_asset_details['metadata']['guid'],
    url=model_asset_details['metadata']['url'],
    name=model_asset_details['entity']['name'],
    description=model_asset_details['entity']['description'],
    deployment_type=model_asset_details['entity']['type'],
    scoring_endpoint = scoring_endpoint
) 

In [ ]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )
os = model_asset_details['entity']['asset_properties']['output_data_schema']
input_s= model_asset_details['entity']['asset_properties']['input_data_schema']
input_schema = SparkStruct(
        type=model_asset_details['entity']['asset_properties']['input_data_schema']['type'],
        fields=model_asset_details['entity']['asset_properties']['input_data_schema']['fields']
    )
output_schema = SparkStruct(
        type=model_asset_details['entity']['asset_properties']['output_data_schema']['type'],
        fields=get_output_schema_fields_for_azure(model_asset_details['entity']['asset_properties']['output_data_schema'], target_field = "Scored Labels", target_role = "decoded-target")
    )
output_schema.to_dict()

In [ ]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk",
        prediction_field='Scored Labels',
        probability_fields=['Scored Probabilities'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=input_schema,
        output_data_schema=output_schema,
        feature_fields=feature_columns,
        categorical_fields=categorical_columns
    )

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=azure_asset,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id

<a id="scoring"></a>
# 4. Scoring and payload logging

### Score the credit risk model and measure response time

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

In [ ]:
import requests
import time
import json

subscription_details=wos_client.subscriptions.get(subscription_id).result.to_dict()
scoring_url = subscription_details['entity']['deployment']['scoring_endpoint']['url']

data = {
            "Inputs": {
                "input1":
                    training_data_pd.sample(2).drop(['Risk'],axis=1).to_dict('records'),
            },
            "GlobalParameters": {
            }
        }

body = str.encode(json.dumps(data))

token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = subscription_details['entity']['deployment']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

## Functions to adjust payload data format for OpenScale 

In [ ]:
def convert_user_input_2_openscale(input_data):
    users_records = input_data['Inputs']['input1']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}

In [ ]:
import numpy as np
def convert_user_output_2_openscale(output_data):
    users_records = output_data['Results']['output1']
    openscale_fields = ["Scored Labels","Scored Probabilities"]
    openscale_values = [[np.double(rec[k]) if k=='Scored Probabilities' else rec[k] for k in openscale_fields ] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}

In [ ]:
openscale_input=convert_user_input_2_openscale(data)
openscale_output=convert_user_output_2_openscale(result)

### Store the request and response in payload logging table

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=openscale_input,
           response=openscale_output,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

In [ ]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

<a id="feedback"></a>
# 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

In [ ]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

In [ ]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

In [ ]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json

### Get feedback logging dataset ID

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
#print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [ ]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)

In [ ]:
# manually records scoring and store as feedback data

# records = [
#     ["no_checking","28","outstanding_credit","appliances","5990","500_to_1000","greater_7","5","male","co-applicant","3","car_other","55","none","free","2","skilled","2","yes","yes","Risk"],
#     ["greater_200","22","all_credits_paid_back","car_used","3376","less_100","less_1","3","female","none","2","car_other","32","none","own","1","skilled","1","none","yes","No Risk"],
#     ["no_checking","39","credits_paid_to_date","vacation","6434","unknown","greater_7","5","male","none","4","car_other","39","none","own","2","skilled","2","yes","yes","Risk"],
#     ["0_to_200","20","credits_paid_to_date","furniture","2442","less_100","unemployed","3","female","none","1","real_estate","42","none","own","1","skilled","1","none","yes","No Risk"],
#     ["greater_200","4","all_credits_paid_back","education","4206","less_100","unemployed","1","female","none","3","savings_insurance","27","none","own","1","management_self-employed","1","none","yes","No Risk"],
#     ["greater_200","23","credits_paid_to_date","car_used","2963","greater_1000","greater_7","4","male","none","4","car_other","46","none","own","2","skilled","1","none","yes","Risk"],
#     ["no_checking","31","prior_payments_delayed","vacation","2673","500_to_1000","1_to_4","3","male","none","2","real_estate","35","stores","rent","1","skilled","2","none","yes","Risk"],
#     ["no_checking","37","prior_payments_delayed","other","6971","500_to_1000","1_to_4","3","male","none","3","savings_insurance","54","none","own","2","skilled","1","yes","yes","Risk"],
#     ["0_to_200","39","prior_payments_delayed","appliances","5685","100_to_500","1_to_4","4","female","none","2","unknown","37","none","own","2","skilled","1","yes","yes","Risk"],
#     ["no_checking","38","prior_payments_delayed","appliances","4990","500_to_1000","greater_7","4","male","none","4","car_other","50","bank","own","2","unemployed","2","yes","yes","Risk"]]

# fields = feature_columns.copy()
# fields.append('Risk')

# payload_scoring =  [{"fields": fields, "values": records}]
#wos_client.data_sets.store_records(feedback_dataset_id, request_body=payload_scoring, background_mode=False)

In [ ]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

# 6. Fairness, Drift monitoring and explanations <a id="monitors"></a>

### Fairness monitor configuration

In [ ]:
wos_client.monitor_instances.show()

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 40
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id

### Payload logging and Run fairness monitor on demand

In [ ]:
import requests
import time
import json

scoring_url = subscription_details['entity']['deployment']['scoring_endpoint']['url']

payload_data = {
            "Inputs": {
                "input1":
                    training_data_pd.sample(40).drop(['Risk'],axis=1).to_dict('records'),
            },
            "GlobalParameters": {
            }
        }

pl_body = str.encode(json.dumps(payload_data))


token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = subscription_details['entity']['deployment']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=pl_body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

### Store the request and response in payload logging table

In [ ]:
openscale_paylod_input=convert_user_input_2_openscale(payload_data)
openscale_payload_output=convert_user_output_2_openscale(result)

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=openscale_paylod_input,
           response=openscale_payload_output,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

In [ ]:
wos_client.data_sets.show_records(payload_data_set_id)

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)

In [ ]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

### Explainability configuration

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

In [ ]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

### Run explanation

In [ ]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

In [ ]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

In [ ]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id, subscription_id=subscription_id).result.to_dict()

### Drift monitor configuration

#### Drift requires a trained model to be uploaded manually for Azure. You can train, create and download a drift detection model using template given ( check for Drift detection model generation) [here](https://github.com/IBM-Watson/aios-data-distribution/blob/master/training_statistics_notebook.ipynb)

In [ ]:
!rm -rf creditrisk_azure_drift_detection_model.tar.gz
!wget -O creditrisk_azure_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/Azure/assets/models/credit_risk/azure_creditrisk_drift_detection_model.tar.gz?raw=true 

In [ ]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_azure_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 40,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

### Run Drift monitor on demand

In [ ]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

## Congratulations!

You have finished the tutorial for IBM Watson OpenScale and Azure Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.



---